Article from Philippe:
https://practicalcheminformatics.blogspot.com/2023/06/getting-real-with-molecular-property.html


In [2]:
# for running in colab:

# !pip install rdkit
# !pip install deepchem


# !wget https://raw.githubusercontent.com/NinaB99/AI-for-Chemistry/main/Data/ADME_public_set_3521.csv
# !wget https://raw.githubusercontent.com/NinaB99/AI-for-Chemistry/main/Data/11095_2013_1222_MOESM2_ESM.csv

# load biogen data
# import pandas as pd

# biogen_data=pd.read_csv("ADME_public_set_3521.csv")
# print(biogen_data.columns)

# #load bioavailabity data
# bio_avail_data = pd.read_csv("11095_2013_1222_MOESM2_ESM.csv",sep=";")
# print(bio_avail_data.columns)

In [2]:
# load biogen data
import pandas as pd

biogen_data=pd.read_csv("../Data/Biogen.csv")
print(biogen_data.columns)

#new data
curated_data = pd.read_csv("../Data/CuratedSol.csv")
print(curated_data.columns)

#load bioavailabity data
bio_avail_data = pd.read_csv("../Data/Bioavailibility.csv")
#print(bio_avail_data.columns)



Index(['Internal ID', 'Vendor ID', 'SMILES', 'CollectionName',
       'LOG HLM_CLint (mL/min/kg)', 'LOG MDR1-MDCK ER (B-A/A-B)',
       'LOG SOLUBILITY PH 6.8 (ug/mL)',
       'LOG PLASMA PROTEIN BINDING (HUMAN) (% unbound)',
       'LOG PLASMA PROTEIN BINDING (RAT) (% unbound)',
       'LOG RLM_CLint (mL/min/kg)', 'MolW(Da)', 'NumHAcceptors', 'NumHDonors',
       'LogP', 'Lipinski_rule'],
      dtype='object')
Index(['ID', 'Name', 'InChI', 'InChIKey', 'SMILES', 'Solubility', 'SD',
       'Ocurrences', 'Group', 'MolWt', 'MolLogP', 'MolMR', 'HeavyAtomCount',
       'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds',
       'NumValenceElectrons', 'NumAromaticRings', 'NumSaturatedRings',
       'NumAliphaticRings', 'RingCount', 'TPSA', 'LabuteASA', 'BalabanJ',
       'BertzCT', 'MolW(Da)', 'LogP', 'Lipinski_rule'],
      dtype='object')


use feed-forward NN using pytorch.
train on solubility from dataset 1 first.
then optimize for bioavailability

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from deepchem.feat import RDKitDescriptors
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
c:\Users\nina\anaconda3\envs\ai_chem\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\nina\anaconda3\envs\ai_chem\Lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loa

In [4]:
#disable warnings from RDKit
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*') 

### Combining datasets

In [5]:
def canonicalize(Dataframe: pd.DataFrame):
    
    """Canonicalizes the SMILES from Dataframe. A column called 'SMILES' is requiered

    Args: Dataframe with 'SMILES' column contaning smiles. 
    """
    
    Dataframe['SMILES'] = Dataframe['SMILES'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x))) #canonicalize smiles from a Dataframe                                          
    

In [7]:

# they are both log so I do not need to log-transform
canonicalize(biogen_data)
canonicalize(curated_data)

#make sure to remove overlaps
import pandas as pd
import numpy as np

# Sample DataFrames (replace with your actual data)
# biogen_data = pd.read_csv('biogen_data.csv')
# curated_data = pd.read_csv('curated_data.csv')

# Rename columns for consistency
biogen_data_new = biogen_data.rename(columns={"LOG SOLUBILITY PH 6.8 (ug/mL)": "Log solubility"})
curated_data_new = curated_data.rename(columns={"Solubility": "Log solubility"})

# Select relevant columns
biogen_data_new = biogen_data_new[["SMILES", "Log solubility"]]
curated_data_new = curated_data_new[["SMILES", "Log solubility"]]

# Combine the DataFrames, prioritizing the curated_data
data = pd.concat([curated_data_new, biogen_data_new])

# Drop duplicate SMILES, keeping the first occurrence (curated_data)
data = data.drop_duplicates(subset="SMILES", keep='first')

# Reset index for the final combined DataFrame
#combined_data = combined_data.reset_index(drop=True)

# Print the combined DataFrame
print(data)



                                                 SMILES  Log solubility
0                   CCCCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-]       -3.616127
1                                  O=C1Nc2cccc3cccc1c23       -3.254767
2                                       O=Cc1ccc(Cl)cc1       -2.177078
3     CC(c1ccccc1)c1cc(C(=O)[O-])c(O)c(C(C)c2ccccc2)...       -3.924409
4     c1cc(N(CC2CO2)CC2CO2)ccc1Cc1ccc(N(CC2CO2)CC2CO...       -4.662065
...                                                 ...             ...
3516            O=C(c1ccc2c(c1)CCCC2)N1CCOCC1c1ccn[nH]1             NaN
3517           O=C(Nc1nc2ccccc2[nH]1)c1ccc(-n2cccc2)cc1             NaN
3518         NC(=O)c1noc([C@@H](CCCC2CCCCC2)CC(=O)NO)n1             NaN
3519         CCCCCCCCc1ccc(CC[C@](N)(CO)COP(=O)(O)O)cc1             NaN
3520  Cc1cccc(/C=N/Nc2cc(N3CCOCC3)n3nc(-c4ccncc4)cc3...             NaN

[13480 rows x 2 columns]


### Data preparation

In [8]:

#### DATA PREPARATION ####
# try with new data

# Function to generate features from SMILES strings using RDKit descriptors
def generate_features(smiles_list):
    featurizer = RDKitDescriptors()
    features = featurizer.featurize(smiles_list)
    # Drop features containing invalid values
    used_features = ~np.isnan(features).any(axis=0)
    features = features[:, ~np.isnan(features).any(axis=0)]
    return features, used_features

#remove nan values from data
data = data.dropna(subset=['Log solubility'])

#get x and y data (x is the molecular descriptors, y is the solubility)
y_data = data["Log solubility"]

print(len(y_data))

# Generate features from SMILES data (get smiles from df)
smiles = data["SMILES"]
X_data, used_features = generate_features(smiles)

#split data into training and validation using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
# Convert y pandas Series to NumPy array
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)

#scale x values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#convert data to pytorch tensors (like numpy arrays but for pytorch)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Reshape the target tensor to match the shape of the output tensor
y_train_tensor = y_train_tensor.view(-1, 1)
# Reshape the target tensor to match the shape of the output tensor
y_test_tensor = y_test_tensor.view(-1, 1)


12147


In [9]:
X_data.shape[1]

198

In [10]:
np.std(y_data)

2.6858044085631407

### Grid search

In [26]:
#### CREATING MODEL ####

#haven't tried it with the newest settings

# Define the neural network architecture
class SolubilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SolubilityPredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define hyperparameters
input_dim = X_data.shape[1]  # Number of molecular descriptors. 201 different ones
hidden_dim = 256   #best according to grid search
output_dim = 1
learning_rate = 0.001 #gets overruled by the grid search
num_epochs = 800
batch_size = 32

#create dataloader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#initialize model, loss function (nn.MSELoss) and optimizer
model = SolubilityPredictor(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

##### Grid-search for best hyperparameters #####
# Define your hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(256,256,256),(512,512,512)],  # Number of neurons in the hidden layer(s)
    'activation': ['relu', 'tanh'],  # Activation function
    'solver': ['adam', 'sgd'],  # Optimization algorithm
    'learning_rate': ['constant', 'adaptive'],  # Learning rate schedule
}

# Create an MLPRegressor object
mlp = MLPRegressor(max_iter=num_epochs, batch_size=batch_size)

# Perform grid search
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found
print("Best Hyperparameters:", grid_search.best_params_)

# Get the best model
model = grid_search.best_estimator_

# Evaluate the best model on the test set
test_score = model.score(X_test, y_test)
print("Test Score (R^2):", test_score)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Hyperparameters: {'activation': 'relu', 'hidden_layer_sizes': (512, 512, 512), 'learning_rate': 'adaptive', 'solver': 'adam'}
Test Score (R^2): 0.1923752586047588


In [27]:
# Evaluate the best model on the test set
test_predictions = model.predict(X_test)

# Calculate additional metrics
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print the results
print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test R^2: {test_r2}")


Test MSE: 0.3812762971349799
Test RMSE: 0.6174757461916864
Test MAE: 0.4129440122100839
Test R^2: 0.1923752586047588


### Without grid search

In [11]:
#### CREATING MODEL ####

# Define the neural network architecture with multiple hidden layers
class SolubilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(SolubilityPredictor, self).__init__()
        layers = []
        previous_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(previous_dim, hidden_dim))
            layers.append(nn.ReLU())
            previous_dim = hidden_dim
        layers.append(nn.Linear(previous_dim, output_dim))
        self.network = nn.Sequential(*layers)
        self.activation = nn.ReLU() #best according to grid search

    def forward(self, x):
        return self.network(x)

# Define hyperparameters
input_dim = X_data.shape[1]  # Number of molecular descriptors
hidden_dim = [512,512,512]  # (512,512,512) Best according to grid search
output_dim = 1
learning_rate = 0.001
num_epochs = 1000
batch_size = 32

# Create DataLoader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function (nn.MSELoss) and optimizer
model = SolubilityPredictor(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
test_score = r2_score(y_test, y_pred)
print("Test Score (R^2):", test_score)

#maybe make plots of training and validation loss to see how it is learning
#can help identify overfitting

Epoch 1/1000, Loss: 943.4128410816193
Epoch 2/1000, Loss: 571.640143930912
Epoch 3/1000, Loss: 495.7608366012573
Epoch 4/1000, Loss: 442.81039038300514
Epoch 5/1000, Loss: 381.6474874615669
Epoch 6/1000, Loss: 353.0723693370819
Epoch 7/1000, Loss: 316.92699748277664
Epoch 8/1000, Loss: 282.5618694126606
Epoch 9/1000, Loss: 256.27133582532406
Epoch 10/1000, Loss: 261.1899362206459
Epoch 11/1000, Loss: 228.9344640672207
Epoch 12/1000, Loss: 226.72700282931328
Epoch 13/1000, Loss: 209.87524117529392
Epoch 14/1000, Loss: 201.9399061203003
Epoch 15/1000, Loss: 182.9896047860384
Epoch 16/1000, Loss: 172.0141593515873
Epoch 17/1000, Loss: 169.30880476534367
Epoch 18/1000, Loss: 156.33979614824057
Epoch 19/1000, Loss: 153.13747150450945
Epoch 20/1000, Loss: 153.09689316153526
Epoch 21/1000, Loss: 144.44090854376554
Epoch 22/1000, Loss: 142.0796439871192
Epoch 23/1000, Loss: 146.01142153143883
Epoch 24/1000, Loss: 133.36737098544836
Epoch 25/1000, Loss: 124.46246548742056
Epoch 26/1000, Loss: 1

In [12]:
model.eval() # Set the model to evaluation mode

# Perform the forward pass to get predictions
with torch.no_grad():
    test_predictions_tensor = model(X_test_tensor)

# Convert predictions back to NumPy array
test_predictions = test_predictions_tensor.numpy().flatten()

# Calculate additional metrics
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print the results
print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test R^2: {test_r2}")

Test MSE: 1.8490358591079712
Test RMSE: 1.3597925901412964
Test MAE: 0.7881057262420654
Test R^2: 0.7358247667031574


### With dropout, weight decay
try dropout rates between 0.2 and 0.5. start with 0.5

In [54]:
# Define the neural network architecture with multiple hidden layers and dropout
class SolubilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(SolubilityPredictor, self).__init__()
        layers = []
        previous_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(previous_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))  # Adding dropout
            previous_dim = hidden_dim
        layers.append(nn.Linear(previous_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Define hyperparameters
input_dim = X_data.shape[1]  # Number of molecular descriptors
hidden_dims = [512, 512, 512]  # Best according to grid search
output_dim = 1
learning_rate = 0.001
num_epochs = 1000
batch_size = 32

# Create DataLoader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function (nn.MSELoss) and optimizer with weight decay (L2 regularization)
model = SolubilityPredictor(input_dim, hidden_dims, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if epoch % 100 == 0:  # Print loss every 100 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).numpy().flatten()

test_mse = mean_squared_error(y_test, y_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test R^2: {test_r2}")


Epoch 1/1000, Loss: 681.7832328081131
Epoch 101/1000, Loss: 111.76737007498741
Epoch 201/1000, Loss: 74.74509523808956
Epoch 301/1000, Loss: 64.90365134179592
Epoch 401/1000, Loss: 56.54144813865423
Epoch 501/1000, Loss: 55.897232234478
Epoch 601/1000, Loss: 51.54749147221446
Epoch 701/1000, Loss: 52.32152093201876
Epoch 801/1000, Loss: 46.192381370812654
Epoch 901/1000, Loss: 48.13287205994129
Test MSE: 1.2193962335586548
Test RMSE: 1.1042627096176147
Test MAE: 0.695110559463501
Test R^2: 0.7752118032844304


### Saving the best model

In [17]:
torch.save(model.state_dict(), 'solubility_model.pth')


## Transfer learning

Need to add extra layers and freeze others? or unfreeze some
can also try to make it classification instead of regression?

In [37]:
data = bio_avail_data

In [38]:

#### DATA PREPARATION ####
# try with new data

#remove nan values from data
data = data.dropna(subset=['%F'])

#get x and y data (x is the molecular descriptors, y is the solubility)
y_data = data["%F"]

print(len(y_data))

#generating features and making sure it's the same features as previous model
smiles = data["Updated SMILES"]
featurizer = RDKitDescriptors()
features = featurizer.featurize(smiles)
X_data = features[:,used_features]


995


In [39]:

print(X_data.shape[1])
if True in np.isnan(X_data):
    print("yes")

198


In [42]:
np.std(y_data)

33.939842104248825

In [43]:


#split data into training and validation using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
# Convert y pandas Series to NumPy array
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)

#scale x values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#convert data to pytorch tensors (like numpy arrays but for pytorch)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Reshape the target tensor to match the shape of the output tensor
y_train_tensor = y_train_tensor.view(-1, 1)
# Reshape the target tensor to match the shape of the output tensor
y_test_tensor = y_test_tensor.view(-1, 1)


In [44]:
#define model again

# Define the neural network architecture with multiple hidden layers
class SolubilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(SolubilityPredictor, self).__init__()
        layers = []
        previous_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(previous_dim, hidden_dim))
            layers.append(nn.ReLU())
            previous_dim = hidden_dim
        layers.append(nn.Linear(previous_dim, output_dim))
        self.network = nn.Sequential(*layers)
        self.activation = nn.ReLU() #best according to grid search

    def forward(self, x):
        return self.network(x)

# Define hyperparameters
input_dim = X_data.shape[1]  # Number of molecular descriptors
hidden_dim = [512,512,512]  # (512,512,512) Best according to grid search
output_dim = 1
learning_rate = 0.001
num_epochs = 1000
batch_size = 32

#loading saved model
model = SolubilityPredictor(input_dim, hidden_dim, output_dim)
model.load_state_dict(torch.load('solubility_model.pth'))

# Optionally, modify the final layer if the output dimension is different

<All keys matched successfully>

In [45]:
class BioavailabilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(BioavailabilityPredictor, self).__init__()
        self.base_model = SolubilityPredictor(input_dim, hidden_dims, output_dim)
        
        # New layers for bioavailability prediction
        self.new_layers = nn.Sequential(
            nn.Linear(output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 1)
        )
    
    def forward(self, x):
        x = self.base_model(x)
        x = self.new_layers(x)
        return x

# Create the new model
bioavailability_model = BioavailabilityPredictor(input_dim, hidden_dim, output_dim)
bioavailability_model.base_model.load_state_dict(model.state_dict())

<All keys matched successfully>

In [46]:
#initially freeze the pre-trained layers
for param in bioavailability_model.base_model.parameters():
    param.requires_grad = False


In [47]:
# train only the new layers added for the task:

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(bioavailability_model.new_layers.parameters(), lr=learning_rate)

# Set the model to training mode
bioavailability_model.train()

# Training loop for new layers
for epoch in range(num_epochs):
    bioavailability_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = bioavailability_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 100 == 0:  # Print loss every 100 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")


Epoch 1/1000, Loss: 1196.350540280342
Epoch 101/1000, Loss: 967.0957604050636
Epoch 201/1000, Loss: 967.3271186351776
Epoch 301/1000, Loss: 965.6701622009277
Epoch 401/1000, Loss: 965.4490968585014
Epoch 501/1000, Loss: 965.169162273407
Epoch 601/1000, Loss: 965.6898025870323
Epoch 701/1000, Loss: 966.0286057591438
Epoch 801/1000, Loss: 965.6987157464027
Epoch 901/1000, Loss: 964.937452673912


In [48]:
bioavailability_model.eval() # Set the model to evaluation mode

# Perform the forward pass to get predictions
with torch.no_grad():
    test_predictions_tensor = bioavailability_model(X_test_tensor)

# Convert predictions back to NumPy array
test_predictions = test_predictions_tensor.numpy().flatten()

# Calculate additional metrics
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print the results
print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test R^2: {test_r2}")

Test MSE: 3727.35498046875
Test RMSE: 61.052066802978516
Test MAE: 50.10044479370117
Test R^2: -2.0400523087652602


In [49]:
#fine-tune it by unfreezing some of the pre-trained layers

# Unfreeze some of the pre-trained layers
for param in bioavailability_model.base_model.network[-3].parameters():  # Unfreeze the second last layer as an example
    param.requires_grad = True
for param in bioavailability_model.base_model.network[-5].parameters():  # Unfreeze the third last layer (optional)
    param.requires_grad = True

# Update the optimizer to include the parameters of the unfrozen layers
optimizer = optim.Adam(bioavailability_model.parameters(), lr=learning_rate / 10)

#maybe 1000 epochs are not needed

# Fine-tuning loop
for epoch in range(num_epochs):
    bioavailability_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = bioavailability_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 100 == 0:  # Print loss every 100 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")


Epoch 1/1000, Loss: 914.3759000301361
Epoch 101/1000, Loss: 66.055250890553
Epoch 201/1000, Loss: 20.716699776239693
Epoch 301/1000, Loss: 12.415890370961279
Epoch 401/1000, Loss: 9.232166432309896
Epoch 501/1000, Loss: 7.7701833446044475
Epoch 601/1000, Loss: 6.864622933673672
Epoch 701/1000, Loss: 6.276480855362024
Epoch 801/1000, Loss: 5.79789643723052
Epoch 901/1000, Loss: 5.296067177318037


In [51]:
bioavailability_model.eval() # Set the model to evaluation mode

# Perform the forward pass to get predictions
with torch.no_grad():
    test_predictions_tensor = bioavailability_model(X_test_tensor)

# Convert predictions back to NumPy array
test_predictions = test_predictions_tensor.numpy().flatten()

# Calculate additional metrics
test_mse = mean_squared_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print the results
print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test R^2: {test_r2}")

Test MSE: 3782.56396484375
Test RMSE: 61.5025520324707
Test MAE: 50.522132873535156
Test R^2: -2.0850813379729947
